In [1]:
import PyPDF2
import os
import re
import requests
import pandas as pd
import numpy as np
from itertools import combinations 
from itertools import permutations
from itertools import chain
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
ps = PorterStemmer()

In [2]:
courseletlist = (
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F175%2F163757167020201210_Liu_crypto_p2p_lending.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F272%2F1654160257Lesson1-1.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F273%2F1654160288Lesson1-2.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F274%2F1654160327Lesson1-3.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F275%2F1654160374Lesson1-4.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F276%2F1654160475Lesson1-5.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F277%2F1654160518Lesson1-6.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F279%2F1654251498Lesson2-1.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F278%2F1654160549Lesson1-7.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F280%2F1654251511Lesson2-2.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F281%2F1654251525Lesson2-3.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F103%2F20210303+IA+METIS+Reinforcement+Learning.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F107%2F1636712642CATE_meets_ML_Presentation.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F120%2F163458263420190429+Hae+Ni+LDA+DTM.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F120%2F163646337920210921+Hae+Ni+LDA.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F121%2F163646358220210708+Hae+Ni+LDA+extensions.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F108%2F163595835420210530+METIS+WANG+Kalman+Filter.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F110%2F1632126441nodalida2021_summaryQuality_slides.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F112%2F163664661320211013+Ren+LI+Hae+Expectile+FRM.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F114%2F1635233254Shapley.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F118%2F1636625638FRM%40EM.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F119%2F163231165920210324+Wan+Hae+Li+k-expectile+clustering.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F130%2F1633104997PAC.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F134%2F163368764620210923+Mer+Hae+GAN+Generative+Adversarial+Networks.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F139%2F163402565120211012+Kho+Hae+Trespassing+random+forests+with+a+pointed+stick+for+self+defence.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F217%2F1644582711Berlin_short_course.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F235%2F1649426301Variable+importance+measures+for+RF+.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F179%2F16376558632021122+SBA+JW+Hae+EPF++Quantinar.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F186%2F1645194357Presentation_Quantinar_with_videos.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F306%2FBarHan2021_talk.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F181%2F163826827620211130+LI+Hae+Case+based+Bancruptcy+prediction.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F224%2F164728525020220305+LI+Electricity+Market+Coupling.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F196%2F1642599075163458186420200403+METIS+Kho+Hae+Spectral+Clustering+course.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F198%2F1642599289163231165920210324+Wan+Hae+Li+k-expectile+clustering+course.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F141%2F163527012720210526+SAE+NAG+HAE+SIZ+Understanding+jumps+in+high+frequency+digital+asset+markets.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F201%2F16426862241636625638FRM%40EM_course.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F202%2F1642686343163774999520210912+Hae+Li+Tao+Dynamic+Crypto+Networks_course.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F189%2F163958119120211130+Hae+Wan+Kot+ComputerMuseum.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F183%2F1643806658KDE+ill-posed+problems.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F167%2F163699399420211115+Liu+Word+Embeddings+2.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F157%2F1642778303introduction_data_science.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F180%2F16377653596.+model+assessment+-+part+4+-+appendix.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F128%2F163707553520210331_METIS_Hel_GANs_for_Time_Series.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F127%2F163458156320190528+Cea+Hae+Scagnostics.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F117%2F163458186420200403+METIS+Kho+Hae+Spectral+Clustering.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F106%2F163458175020200914+Hae+DS2+Data+Science+%26+Digital+Society.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F190%2F1640038524Instruction+for+Creating+Quantlets.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F144%2F1636624210NNCSR_Slides.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F184%2F163888970320211207+Zin+Reu+Hae+USC+Quantinar+40+min+PDF.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F133%2F163707963820210525_Hae_Xia_Crypto_Indices-2.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F129%2F163458163120200915+Kim+Hae+Tri+VCRIX.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F123%2F163664570620210922+Mat+Pac+Hae+guide+hedging+CC.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F122%2F163283000220210923+Cul+Hae+Pet+Xia+Cryptocurrency+as+an+asset+class.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F116%2F163774999520210912+Hae+Li+Tao+Dynamic+Crypto+Networks.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F113%2F1632580703FRM+for+Cryptos.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F101%2F163170747120210914+Reu+DSF+Digital+Surrogate+Finance+Doc.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F97%2F163458105020210808+METIS+Win+Pricing+Kernel+Risk+Premium.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F96%2F163299370720210908_CRC21_Hae_Rodeo_or_Ascot.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F96%2F1635155323202109_RoA.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F79%2F164604019720210502+Hae+Har+Reu+Understanding+CCs.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F142%2F163627928020211107+Hae+Iva+Mat+Delaunay+Triangulation_A_Shape.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F135%2F1649084960Chapter+1.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F138%2F1649094430Chapter+4.pdf',
# problematic:
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F197%2F1642599236164171878820211207+Hae+Zin+Hierarchical+Clustering+course.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F241%2F1650632942Biographical+Background+Information.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F210%2F164390414020220130+METIS+Gua+Hae+Model+Selection+Criteria.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F194%2F164171878820211207+Hae+Zin+Hierarchical+Clustering.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F170%2F163709451820211117+Hae+Qia+Network+Centrality.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F136%2F1649094328Chapter+2.pdf',
'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F137%2F1633950248Chapter+3.pdf'
)

ids = (["id-{}".format(i+1) for i in range(len(courseletlist))])


stopwords_list = {"i","me","my","myself","we","our","ours","ourselves","you","you're","you've","you'll","you'd","your","yours","yourself","yourselves","he","him","his","himself","she","she's","her","hers","herself","it","it's","its","itself","they","them","their","theirs","themselves","what","which","who","whom","this",'that',"that'll","these","those","am","is","are","was","were","be","been","being","have","has","had","having","do","does","did","doing","a","an","the","and","but","if","or","because","as","until","while","of","at","by","for","with","about","against","between","into","through","during","before","after","above","below","to","from","up","down","in","out","on","off","over","under","again","further","then","once","here","there","when","where","why","how","all","any","both","each","few","more","most","other","some","such","no","nor","not","only","own","same","so","than","too","very","s","t","can","will","just","don","don't","should","should've","now","d","ll","m","o","re","ve","y","ain","aren","aren't","couldn","couldn't","didn","didn't","doesn","doesn't","hadn","hadn't","hasn","hasn't","haven","haven't","isn","isn't","ma","mightn","mightn't","mustn","mustn't","needn","needn't","shan","shan't","shouldn","shouldn't","wasn","wasn't","weren","weren't","won","won't","wouldn","wouldn't"}

id_matrix = pd.DataFrame({'id': ids, 'url': courseletlist})

In [3]:
id_matrix = id_matrix.loc[1:32]

In [3]:
def download_pdf(file_name, url):
    '''Download a PDF file with an URL (Step 1)'''
    # Define HTTP Headers
    headers = {"User-Agent": "Chrome/51.0.2704.103"}
    # Download image
    response = requests.get(url, headers=headers)
    # if response is OK download the PDF and store it, else write the status
    if response.status_code == 200:
        with open(file_name, "wb") as f:
            f.write(response.content)
    else:
        print(response.status_code)
    return

In [4]:
def create_string(file_name):
    '''Transform a PDF file to a list of string pages (Step 2)'''
    # opening the file
    imported_pdf = open(file_name, 'rb')
    # convert PDF to readable file
    transformed_pdf = PyPDF2.PdfFileReader(imported_pdf)
    # get number of pages
    totalpages = transformed_pdf.numPages
    # read the data and store in a list
    pdf_output = [transformed_pdf.getPage(i) for i in range(totalpages)]
    # extract result
    pdf_output = [pdf_output[i].extractText() for i in range(totalpages)]
    return pdf_output, totalpages 

In [ ]:
# # old
# def cleaning(file_name):
#     '''Initial PDF cleaning procedure (Step 3)'''
#     pdf_output, totalpages = create_string(file_name)
#     # # cleaning URLs
#     pdf_output = [re.sub(pattern = "http[^ ]*", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
#     # # cleaning symbols
#     pdf_output = [re.sub(pattern = "\\n", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
#     pdf_output = [re.sub(pattern = "\W|\d", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
#     pdf_output = [re.sub(pattern = "[^a-zA-Z]", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
#     # # cleaning multispaces
#     pdf_output = [re.sub(pattern = "\s{2,}", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
#     # # cleaning out 1-2-worders
#     pdf_output = [re.sub(pattern = " .{1,2} ", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
#     pdf_output = [re.sub(pattern = " .{1,2} ", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
#     pdf_output = [re.sub(pattern = " .{1,2} ", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
#     # # lower-casing
#     pdf_output = [pdf_output[i].lower() for i in range(totalpages)]
#     pdf_output =  [[ps.stem(word) for word in sentence.split(" ")] for sentence in pdf_output]
#     return pdf_output, totalpages

In [5]:
def cleaning(file_name):
    '''Initial PDF cleaning procedure (Step 3)'''
    pdf_output, totalpages = create_string(file_name)
    # # cleaning URLs
    pdf_output = [re.sub(pattern = "http[^ ]*", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    # # cleaning symbols
    pdf_output = [re.sub(pattern = "\\n", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    pdf_output = [re.sub(pattern = "\W|\d", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    pdf_output = [re.sub(pattern = "[^a-zA-Z]", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    # # cleaning multispaces
    pdf_output = [re.sub(pattern = "\s{2,}", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    # # cleaning out 1-2-worders
    pdf_output = [re.sub(pattern = " .{1,2} ", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    pdf_output = [re.sub(pattern = " .{1,2} ", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    pdf_output = [re.sub(pattern = " .{1,2} ", repl = " ", string = pdf_output[i]) for i in range(totalpages)]
    # # lower-casing
    pdf_output = [pdf_output[i].lower() for i in range(totalpages)]
    pdf_output = [[ps.stem(word) for word in sentence.split(" ")] for sentence in pdf_output]
    pdf_output = [' '.join(pdf_output[i]) for i in range(len(pdf_output))]
    return pdf_output, totalpages

In [6]:
def word_lists(file_name):
    '''Creating the base one-word, two-word and three-word lists, the permutation lists for two- and three-word lists (Step 4)'''
    pdf_output, totalpages = cleaning(file_name)
    # split to a list
    word_list = [pdf_output[i].split(" ") for i in range(totalpages)]
    # stemming
    word_list_stemmed = [[ps.stem(word_list[i][j]) for j in range(len(word_list[i]))] for i in range(totalpages)]    
    word_list_stemmed = pd.DataFrame(word_list_stemmed)
    # one-word section
    one_word_list = [word_list_stemmed.iloc[j, i] for j in range(totalpages) for i in range(word_list_stemmed.shape[1])]
  
    one_word_list = [x for x in one_word_list if x not in stopwords_list]

    # two-word section
    two_word_list = [[word_list_stemmed.iloc[j, i], word_list_stemmed.iloc[j, i+1]] for j in range(totalpages)  for i in range(word_list_stemmed.shape[1] - 1)]
    two_word_permutation_list = [[p for p in permutations(two_word_list[k])][1:] for k in range(len(two_word_list))]
    two_word_permutation_set = set(list(chain(*two_word_permutation_list)))
    two_word_permutation_set = pd.DataFrame(two_word_permutation_set)

    # three-word section
    three_word_list = [[word_list_stemmed.iloc[j, i], word_list_stemmed.iloc[j, i+1], word_list_stemmed.iloc[j, i+2]] for j in range(totalpages) for i in range(word_list_stemmed.shape[1] - 2)]
    three_word_permutation_list = [[p for p in permutations(three_word_list[k])][1:] for k in range(len(three_word_list))]
    three_word_permutation_set = set(list(chain(*three_word_permutation_list)))
    three_word_permutation_set = pd.DataFrame(three_word_permutation_set)

    return word_list_stemmed, one_word_list, two_word_list, two_word_permutation_list, two_word_permutation_set, three_word_list, three_word_permutation_list, three_word_permutation_set

In [7]:
def occurance_one_matrix_creator(file_name):
    '''Creating the occurrance matrix for one-word combinations (Step 7)'''
    word_list_stemmed, one_word_list, two_word_list, two_word_permutation_list, two_word_permutation_set, three_word_list, three_word_permutation_list, three_word_permutation_set = word_lists(file_name)
    # copying the data
    words = one_word_list.copy()
    # creating the three-word combinations as one string
    words = [x for x in words if x != ""]
    words = [x for x in words if x != " "]    
    # crating the dictionary
    dictionary_one_word = {}
    
    # counting word occurances
    for word in words:
        if word in dictionary_one_word:
            dictionary_one_word[word] = dictionary_one_word[word] + 1
        else:
            dictionary_one_word[word] = 1
    dictionary_one_word = list(dictionary_one_word)
    dictionary_one_word = list(filter(None, dictionary_one_word))
    return dictionary_one_word

In [8]:
download_pdf(ids[0], courseletlist[0])

pdf_output, totalpages = cleaning(ids[0])

word_list_stemmed, one_word_list, two_word_list, two_word_permutation_list, two_word_permutation_set, three_word_list, three_word_permutation_list, three_word_permutation_set = word_lists(ids[0])

dictionary_one_word = occurance_one_matrix_creator(ids[0])

combined_pdf = [' '.join(pdf_output)]

In [9]:
for i in range(1, 70):
    print(i)
    download_pdf(ids[i], courseletlist[i])
    pdf_output, totalpages = cleaning(ids[i])
    word_list_stemmed, one_word_list, two_word_list, two_word_permutation_list, two_word_permutation_set, three_word_list, three_word_permutation_list, three_word_permutation_set = word_lists(ids[i])
    dictionary_one_word_0 = occurance_one_matrix_creator(ids[i])
    [dictionary_one_word.append(j) for j in dictionary_one_word_0]
    combined_pdf.append(' '.join(pdf_output))


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40


Superfluous whitespace found in object header b'1' b'0'
Superfluous whitespace found in object header b'2' b'0'
Superfluous whitespace found in object header b'3' b'0'
Superfluous whitespace found in object header b'15' b'0'
Superfluous whitespace found in object header b'44' b'0'
Superfluous whitespace found in object header b'48' b'0'
Superfluous whitespace found in object header b'52' b'0'
Superfluous whitespace found in object header b'55' b'0'
Superfluous whitespace found in object header b'14' b'0'
Superfluous whitespace found in object header b'11' b'0'
Superfluous whitespace found in object header b'10' b'0'
Superfluous whitespace found in object header b'9' b'0'
Superfluous whitespace found in object header b'13' b'0'
Superfluous whitespace found in object header b'12' b'0'
Superfluous whitespace found in object header b'43' b'0'
Superfluous whitespace found in object header b'24' b'0'
Superfluous whitespace found in object header b'23' b'0'
Superfluous whitespace found in obj

41


Superfluous whitespace found in object header b'1' b'0'
Superfluous whitespace found in object header b'2' b'0'
Superfluous whitespace found in object header b'3' b'0'
Superfluous whitespace found in object header b'23' b'0'
Superfluous whitespace found in object header b'35' b'0'
Superfluous whitespace found in object header b'46' b'0'
Superfluous whitespace found in object header b'49' b'0'
Superfluous whitespace found in object header b'52' b'0'
Superfluous whitespace found in object header b'71' b'0'
Superfluous whitespace found in object header b'83' b'0'
Superfluous whitespace found in object header b'22' b'0'
Superfluous whitespace found in object header b'11' b'0'
Superfluous whitespace found in object header b'10' b'0'
Superfluous whitespace found in object header b'9' b'0'
Superfluous whitespace found in object header b'20' b'0'
Superfluous whitespace found in object header b'19' b'0'
Superfluous whitespace found in object header b'18' b'0'
Superfluous whitespace found in obj

42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69


In [10]:
vectorizer_one = TfidfVectorizer(smooth_idf=True, sublinear_tf=True, use_idf=True, lowercase = False, stop_words = stopwords_list, ngram_range=(1,1))
X_one = vectorizer_one.fit_transform(combined_pdf)

vectorizer_two = TfidfVectorizer(smooth_idf=True, sublinear_tf=True, use_idf=True, lowercase = False, stop_words = stopwords_list, ngram_range=(2,2))
X_two = vectorizer_two.fit_transform(combined_pdf)

vectorizer_three = TfidfVectorizer(smooth_idf=True, sublinear_tf=True, use_idf=True, lowercase = False, stop_words = stopwords_list, ngram_range=(3,3))
X_three = vectorizer_three.fit_transform(combined_pdf)

In [11]:
# THIS WORKS

tfidf_vectorizer=TfidfVectorizer(smooth_idf=True, sublinear_tf=True, use_idf=True, lowercase = False, stop_words = stopwords_list, ngram_range=(1,1)) 

tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(combined_pdf)

matrix_tfidfvectorizer=tfidf_vectorizer_vectors

df = pd.DataFrame(matrix_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names_out(), columns=id_matrix['id']) 
df

id,id-1,id-2,id-3,id-4,id-5,id-6,id-7,id-8,id-9,id-10,...,id-61,id-62,id-63,id-64,id-65,id-66,id-67,id-68,id-69,id-70
aaab,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaac,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaacchicbvfna,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaacehicbvfnbxmxepuuxyv,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aaack,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zyt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zytdnt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zytjtvt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
zzl,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# # The tuple represents: (document_id, token_id)
# # The value following the tuple represents the tf-idf score of a given token in a given document
# # The tuples that are not there have a tf-idf score of 0
# # vectorizer_one.get_feature_names_out()[5278]

# print(X_one)

In [12]:
xx_one = pd.DataFrame(X_one.toarray(), columns = 
vectorizer_one.get_feature_names_out())

xx_two = pd.DataFrame(X_two.toarray(), columns = 
vectorizer_two.get_feature_names_out())

xx_three = pd.DataFrame(X_three.toarray(), columns = 
vectorizer_three.get_feature_names_out())

In [ ]:
X_one.toarray()

# SVD PART

In [13]:
lsa_obj = TruncatedSVD(n_components=30)
tfidf_lsa_data_one = lsa_obj.fit_transform(X_one)
Sigma_one = lsa_obj.singular_values_
V_T_one = lsa_obj.components_.T

In [ ]:
lsa_obj = TruncatedSVD(n_components=30)
tfidf_lsa_data_two = lsa_obj.fit_transform(X_two)
Sigma_two = lsa_obj.singular_values_
V_T_two = lsa_obj.components_.T

In [ ]:
lsa_obj = TruncatedSVD(n_components=30)
tfidf_lsa_data_three = lsa_obj.fit_transform(X_three)
Sigma_three = lsa_obj.singular_values_
V_T_three = lsa_obj.components_.T

In [14]:
# generates U
tfidf_lsa_data_one

array([[ 0.33959657, -0.13252616, -0.05815999, ...,  0.17129969,
         0.3206116 , -0.08372254],
       [ 0.25662924, -0.24386218, -0.27344773, ...,  0.08320862,
         0.06140737,  0.13423528],
       [ 0.29381235, -0.24360558, -0.28536261, ...,  0.02073898,
         0.06572342,  0.04026229],
       ...,
       [ 0.33106741,  0.02280927,  0.07490754, ...,  0.04092529,
         0.13280786,  0.14077805],
       [ 0.2137848 ,  0.17835266, -0.10624062, ..., -0.0534932 ,
         0.0832899 ,  0.0930224 ],
       [ 0.24011993,  0.21843181, -0.10298943, ...,  0.0227846 ,
        -0.02774666, -0.05101788]])

In [16]:
# generate Sigma.
# The singular values corresponding to each of the selected components. The singular values are equal to the 2-norms of the n_components variables in the lower-dimensional space.
Sigma_one

array([2.68216527, 1.51055267, 1.47513271, 1.37237082, 1.32592356,
       1.31453848, 1.2952371 , 1.26685274, 1.24929095, 1.20949671,
       1.1627485 , 1.14200215, 1.13024925, 1.11323477, 1.05442694,
       1.05194955, 1.04532784, 1.01175333, 1.00814593, 0.99494734,
       0.99105862, 0.98262074, 0.98170026, 0.97602317, 0.97102693,
       0.96989151, 0.95616646, 0.94797109, 0.94196336, 0.93425204])

In [17]:
# The right singular vectors of the input data.
lsa_obj.components_

array([[ 6.91590711e-03,  1.10663720e-03,  8.56310668e-04, ...,
         2.74643419e-03,  8.56310668e-04,  1.56437959e-03],
       [-2.04410374e-03,  2.95229871e-04,  2.04274091e-04, ...,
        -4.17920678e-03,  2.04274091e-04, -2.22267046e-04],
       [-1.39331620e-02, -1.46316662e-03, -1.12264527e-03, ...,
         7.41821542e-03, -1.12264527e-03, -1.00700063e-04],
       ...,
       [ 2.21854721e-03,  5.15023594e-03, -1.22828517e-04, ...,
        -9.93843487e-05, -1.22828517e-04,  1.60894141e-02],
       [ 2.46030891e-03,  1.20583085e-03,  4.40848403e-04, ...,
         2.61058391e-03,  4.40848403e-04, -3.76358168e-05],
       [ 5.98088826e-03, -2.72445927e-03, -4.68474192e-03, ...,
         1.15396922e-03, -4.68474192e-03,  1.49954118e-02]])

In [19]:
# generates Sgenerates V_T
V_T_one

array([[ 6.91590711e-03, -2.04410374e-03, -1.39331620e-02, ...,
         2.21854721e-03,  2.46030891e-03,  5.98088826e-03],
       [ 1.10663720e-03,  2.95229871e-04, -1.46316662e-03, ...,
         5.15023594e-03,  1.20583085e-03, -2.72445927e-03],
       [ 8.56310668e-04,  2.04274091e-04, -1.12264527e-03, ...,
        -1.22828517e-04,  4.40848403e-04, -4.68474192e-03],
       ...,
       [ 2.74643419e-03, -4.17920678e-03,  7.41821542e-03, ...,
        -9.93843487e-05,  2.61058391e-03,  1.15396922e-03],
       [ 8.56310668e-04,  2.04274091e-04, -1.12264527e-03, ...,
        -1.22828517e-04,  4.40848403e-04, -4.68474192e-03],
       [ 1.56437959e-03, -2.22267046e-04, -1.00700063e-04, ...,
         1.60894141e-02, -3.76358168e-05,  1.49954118e-02]])

In [ ]:
print(lsa_obj.explained_variance_ratio_)

print(lsa_obj.explained_variance_ratio_.sum())

print(lsa_obj.singular_values_)

print(lsa_obj.feature_names_in_)

In [ ]:
# import seaborn as sns
# ax = sns.barplot(x=lsa_obj.feature_names_in_[0:20], y = Sigma[0:20])
# ax.set_xticklabels(ax.get_xticklabels(),rotation = 90)

In [ ]:
# term_topic_matrix = pd.DataFrame(data=lsa_term_topic, 
#                                  index = eda_train.columns, 
#                                  columns = [f'Latent_concept_{r}' for r in range(0,V_T.shape[1])])



# Input search

In [21]:
input_one = 'pricing'
input_one = input_one.split(' ')
input_one = [i for i in input_one if i not in stopwords_list]
input_one = [ps.stem(i) for i in input_one]

input_two = 'arbitrage market'
input_two = input_two.split(' ')
input_two = [i for i in input_two if i not in stopwords_list]
input_two = [ps.stem(i) for i in input_two]
input_two = [' '.join(input_two)]

input_three = 'statistics of financial markets'
input_three = input_three.split(' ')
input_three = [i for i in input_three if i not in stopwords_list]
input_three = [ps.stem(i) for i in input_three]
input_three = [' '.join(input_three)]

In [23]:
if input_one[0] in list(xx_one.columns):
    ranked_indexes_one = xx_one[input_one].sort_values(by = input_one, ascending=False).index
    ranked_indexes_one = list(ranked_indexes_one[0:5])
    output_one = [courseletlist[i] for i in ranked_indexes_one]
    print(output_one)
else: 
    print('not present')

['https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F186%2F1645194357Presentation_Quantinar_with_videos.pdf', 'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F175%2F163757167020201210_Liu_crypto_p2p_lending.pdf', 'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F272%2F1654160257Lesson1-1.pdf', 'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F281%2F1654251525Lesson2-3.pdf', 'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F103%2F20210303+IA+METIS+Reinforcement+Learning.pdf']


In [24]:
if input_two[0] in list(xx_two.columns):
    ranked_indexes_two = xx_two[input_two].sort_values(by = input_two, ascending=False).index
    ranked_indexes_two = list(ranked_indexes_two[0:5])
    output_two = [courseletlist[i] for i in ranked_indexes_two]
    print(output_two)
else: 
    print('not present')

['https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F175%2F163757167020201210_Liu_crypto_p2p_lending.pdf', 'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F272%2F1654160257Lesson1-1.pdf', 'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F196%2F1642599075163458186420200403+METIS+Kho+Hae+Spectral+Clustering+course.pdf', 'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F181%2F163826827620211130+LI+Hae+Case+based+Bancruptcy+prediction.pdf', 'https://quantinar.s3.eu-west-3.amazonaws.com/courselet_components%2F306%2FBarHan2021_talk.pdf']


In [25]:
if input_three[0] in list(xx_three.columns):
    ranked_indexes_three = xx_three[input_three].sort_values(by = input_three, ascending=False).index
    ranked_indexes_three = list(ranked_indexes_three[0:5])
    output_three = [courseletlist[i] for i in ranked_indexes_three]
    print(output_three)
else: 
    print('not present')

not present
